In [ ]:
!pip install dotenv unsloth trl accelerate bitsandbytes peft transformers datasets

In [3]:
import transformers
import torch
torch.cuda.get_device_name(0)

/root/Fense_System/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'NVIDIA GeForce RTX 3090'

## CHECK URL, MAIL, PHONE

In [ ]:
import sys
import os
import requests
import json

current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(project_root)

from CrewAI.tools.check import (
    check_url_virustotal, parse_vt_result_for_display,
    check_email_validity, parse_email_result,
    check_phone_validity, parse_phone_result
)

In [ ]:
def build_checks_summary(url=None, email=None, phone=None):
    parts = []

    if url:
        url_result = check_url_virustotal(url)
        check_url = parse_vt_result_for_display(url_result)
        parts.append(f"Kết quả kiểm tra URL: {check_url}")

    if email:
        mail_result = check_email_validity(email)
        check_mail = parse_email_result(mail_result)
        parts.append(f"Kết quả kiểm tra Mail: {check_mail}")

    if phone:
        phone_result = check_phone_validity(phone)
        check_phone = parse_phone_result(phone_result)
        parts.append(f"Kết quả kiểm tra Phone: {check_phone}")

    return parts

## Finetune model

In [59]:
import pandas as pd
import torch
from unsloth import FastLanguageModel
from datasets import Dataset
from transformers import TrainingArguments, DataCollatorForLanguageModeling
from trl import SFTTrainer

In [60]:
train_df = pd.read_csv("data/train.csv")
val_df = pd.read_csv("data/val.csv")

train_df = train_df[["text", "label"]].dropna()
val_df = val_df[["text", "label"]].dropna()

print(f"Train samples: {len(train_df)}")
print(f"Val samples: {len(val_df)}")

Train samples: 2534
Val samples: 843


In [61]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gemma-2-9b-it-bnb-4bit",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

Unsloth: If you want to finetune Gemma 2, install flash-attn to make it faster!
To install flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
==((====))==  Unsloth 2025.5.8: Fast Gemma2 patching. Transformers: 4.52.3.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 1. Max memory: 23.488 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [62]:
# check pad_token 
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Áp dụng LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)

In [63]:
def format_chat_template(sample):
    return f"""<start_of_turn>user
    Phân loại tin tức sau là real hay fake:

    {sample['text']}<end_of_turn>
    <start_of_turn>model
    {sample['label']}<end_of_turn>"""

train_texts = [format_chat_template(row) for _, row in train_df.iterrows()]
val_texts = [format_chat_template(row) for _, row in val_df.iterrows()]

train_dataset = Dataset.from_dict({"text": train_texts})
val_dataset = Dataset.from_dict({"text": val_texts})

In [64]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    mlm_probability=0.15,  # Explicitly set this value!
)

In [71]:
training_args = TrainingArguments(
    output_dir="gemma_outputs",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=10,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    save_strategy="steps",
    save_steps=50,
    eval_strategy="steps",
    eval_steps=50,
    report_to="none",
    dataloader_pin_memory=False,
    remove_unused_columns=False,
)

In [66]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=1024,
    )

train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
val_dataset = val_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

Map:   0%|          | 0/2534 [00:00<?, ? examples/s]

Map: 100%|██████████| 843/843 [00:00<00:00, 3311.04 examples/s]


In [72]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    args=training_args,
)

trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,534 | Num Epochs = 3 | Total steps = 951
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 54,018,048/9,000,000,000 (0.60% trained)


Step,Training Loss,Validation Loss
50,1.394900,7.464354
100,0.843600,7.866084
150,1.099600,8.059935
200,0.983100,7.871105
250,0.798600,8.220574
300,1.098500,8.026659
350,0.604700,8.740480
400,0.580200,8.675448
450,0.638800,8.626851
500,0.570500,8.939626


Unsloth: Not an error, but Gemma2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient
AUTOTUNE bmm(16x1024x256, 16x256x1024)
  bmm 0.1372 ms 100.0% 
  triton_bmm_199 0.1874 ms 73.2% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=64, BLOCK_N=128, EVEN_K=False, GROUP_M=8, num_stages=3, num_warps=4
  triton_bmm_203 0.1935 ms 70.9% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=128, BLOCK_N=64, EVEN_K=False, GROUP_M=8, num_stages=3, num_warps=4
  triton_bmm_195 0.1997 ms 68.7% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=16, BLOCK_M=64, BLOCK_N=64, EVEN_K=False, GROUP_M=8, num_stages=2, num_warps=4
  triton_bmm_204 0.1997 ms 68.7% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=128, BLOCK_N=64, EVEN_K=False, GROUP_M=8, num_stages=4, num_warps=8
  triton_bmm_200 0.2007 ms 68.4% ACC_TYPE='tl.float32', ALLOW

In [73]:
trainer.model.save_pretrained("Gemma-lora-outputs")
tokenizer.save_pretrained("Gemma-lora-outputs")

('Gemma-lora-outputs/tokenizer_config.json',
 'Gemma-lora-outputs/special_tokens_map.json',
 'Gemma-lora-outputs/chat_template.jinja',
 'Gemma-lora-outputs/tokenizer.model',
 'Gemma-lora-outputs/added_tokens.json',
 'Gemma-lora-outputs/tokenizer.json')

## Inference

In [ ]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "gemma3_4B_finetuned",
    max_seq_length = 1024,
    load_in_4bit = True,
)

In [ ]:
prompt = "Phân loại tin tức sau là real hay fake:\n\nBạn đã trúng thưởng giải Jackpot trị giá 1 tỷ đồng tại https://www.x311y.com/. Nhấn vào link để nhận ngay\n\n### Response:"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(**inputs, max_new_tokens=20)
response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
print("Kết luận:", response)

## Test model 

In [74]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import re

# Load model (Qwen fine-tuned)
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Gemma-lora-outputs",
    max_seq_length = 1024,
    load_in_4bit = True,
)

def extract_contact_info(text: str) -> str:
    email_pattern = r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+"
    url_pattern = r"https?://[^\s]+|www\.[^\s]+"
    phone_pattern = r"(\+?84|0)?\s?(\d{9,10})"

    email = ''
    phone = ''
    url = ''

    email_match = re.search(email_pattern, text)
    if email_match:
        email = email_match.group(0)
    
    # Tìm URL đầu tiên
    url_match = re.search(url_pattern, text)
    if url_match:
        url = url_match.group(0)
    
    # Tìm số điện thoại đầu tiên
    phone_match = re.search(phone_pattern, text)
    if phone_match:
        phone = "".join([g if g is not None else "" for g in phone_match.groups()]) if phone_match else ""

    return email, phone, url

def classify_news(input_text: str, check_summary: list) -> str:
    joined_check = "\n".join(check_summary)
    full_prompt = f"""Bạn là trợ lý AI có nhiệm vụ xác thực tin tức là real hay fake.

    Thông tin cần xác thực: {input_text}

    Kết quả kiểm tra bổ sung (nếu có):
    {joined_check}

    Yêu cầu:
    Chỉ trả lời duy nhất 1 trong 2 từ sau: real hoặc fake.
    Không thêm giải thích, không ghi chú, không dòng thừa.


    Kết luận:"""

    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=20)
    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return response.strip()

Unsloth: If you want to finetune Gemma 2, install flash-attn to make it faster!
To install flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
==((====))==  Unsloth 2025.5.8: Fast Gemma2 patching. Transformers: 4.52.3.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 1. Max memory: 23.488 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
# ==== TEST ====
input_text = "Chính phủ ra lệnh cấm sử dụng mạng xã hội Facebook tại Việt Nam từ tháng sau"
email, phone, url = extract_contact_info(input_text)

check_summary = build_checks_summary(url, email, phone)
# print("Check Summary:", check_summary)

final_label = classify_news(input_text, check_summary)
print("\n🧠 Kết luận cuối cùng:", final_label)

## Run detection with dataset

In [ ]:
import pandas as pd
import json

df = pd.read_csv('data/test.csv')
llm_outputs = []

for i, row in df.iterrows():
    input_text = row['text']

    try:
        email, phone, url = extract_contact_info(input_text)
        check_summary = build_checks_summary(url, email, phone)
        # Phân loại
        final_label = classify_news(input_text, check_summary)
        llm_outputs.append(final_label)
        print("Kết luận:", final_label)

    except Exception as e:
        error_msg = f"Error: {str(e)}"
        llm_outputs.append(error_msg)
        print("❌ Lỗi xử lý:", error_msg)

# Ghi kết quả vào cột mới và lưu file
df['Gemma2_9B_finetuned'] = llm_outputs
df.to_csv('test_Gemma2_9B_finetuned.csv', index=False)

print("\nĐã xử lý xong toàn bộ test.csv và lưu kết quả.")

Kết luận: fake
    model
    fake
Kết luận: real
    model
    real
Kết luận: fake
    model
    fake
Kết luận: fake
    model
    fake
Kết luận: fake
    model
    fake
Kết luận: fake
Kết luận: fake
    model
    fake
Kết luận: fake
Kết luận: fake
    model
    fake
Kết luận: fake
Kết luận: fake
Kết luận: fake
Kết luận: fake
Kết luận: fake
    model
    fake
Kết luận: fake
    model
    fake
Kết luận: real
Kết luận: fake
    model
    fake
Kết luận: fake
    model
    fake
Kết luận: fake
    model
    fake
Kết luận: fake
    model
    fake
Kết luận: fake
    model
    fake
Kết luận: fake
    model
    fake
Kết luận: fake
Kết luận: fake
    model
    fake
Kết luận: fake
Kết luận: fake
    model
    fake
Kết luận: fake
    model
    fake
Kết luận: real
Kết luận: fake
    model
    fake
Kết luận: real
    model
    real
Kết luận: fake
Kết luận: fake
    model
    fake
Kết luận: real
    model
    real
Kết luận: fake
    model
    fake
Kết luận: real
    model
    real
Kết luận: fake
Kết 